In [1]:
!pip install azure-storage-file-datalake azure-identity azure-storage-blob

     |████████████████████████████████| 188 kB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 341 kB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 85 kB 14.3 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 33.9 MB/s eta 0:00:01
     |████████████████████████████████| 401 kB 46.8 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 43.3 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 27.9 MB/s eta 0:00:01
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os, uuid, sys
from azure.identity import ClientSecretCredential
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings

def initialize_storage_account_ad(storage_account_name, client_id, client_secret, tenant_id):
    
    try:  
        global service_client

        credential = ClientSecretCredential(tenant_id, client_id, client_secret)

        service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=credential)
    
    except Exception as e:
        print(e)
        
def download_file_from_directory():
    try:
        file_system_client = service_client.get_file_system_client(file_system="mycontainer")
        directory_client = file_system_client.get_directory_client("sample")
        
        local_file = open("iris.data",'wb')

        file_client = directory_client.get_file_client("iris.data")

        download = file_client.download_file()

        downloaded_bytes = download.readall()

        local_file.write(downloaded_bytes)

        local_file.close()

    except Exception as e:
     print(e)

# Enter service principal credentials with access to storage account
storage_account_name = ""
client_id = ""
client_secret = ""
tenant_id = ""

# Initialize and download dataset
initialize_storage_account_ad(storage_account_name, client_id, client_secret, tenant_id)
download_file_from_directory()

In [61]:
import pandas as pd
from sklearn import svm
from joblib import dump, load
from sklearn.linear_model import LogisticRegression


# Read training data set
train_df = pd.read_csv("iris.data", header=None, names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
y = pd.factorize(train_df["class"])[0]

train_df.pop("class")
X = train_df.values

# Train model
clf = LogisticRegression()
clf.fit(X,y)

# Test model
print(X[0:2])
print(clf.predict(X[0:2]))

# Save model to local disk
dump(clf, 'model.joblib')

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]]
[0 0]


['model.joblib']

In [62]:
# Save model to Azure blob storage

import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

container_name = "mycontainer"
local_file_name = "model.joblib"
upload_path = "sklearn/model.joblib"

try:
    connect_str = ""
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=upload_path)

    print("\nUploading to Azure Storage as blob:\n\t" + upload_path)

    # Upload the created file
    with open(local_file_name, "rb") as data:
        blob_client.upload_blob(data)
    
except Exception as ex:
    print('Exception:')
    print(ex)


Uploading to Azure Storage as blob:
	sklearn/model.joblib


In [4]:
%%bash

curl -O https://mirror.openshift.com/pub/openshift-v4/clients/oc/4.6/linux/oc.tar.gz
tar xzf oc.tar.gz
cp oc /opt/app-root/bin/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23.2M  100 23.2M    0     0  38.9M      0 --:--:-- --:--:-- --:--:-- 38.9M


In [63]:
%%bash

# Configure anonymous access to blob container
# https://docs.microsoft.com/en-us/azure/storage/blobs/anonymous-read-access-configure?tabs=portal

# Test oc
oc login --server https://openshift.default.svc.cluster.local --insecure-skip-tls-verify --token=

# Run model in Seldon
oc apply -n odh -f - <<EOF
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: example
spec:
  name: iris
  predictors:
    - graph:
        children: []
        implementation: SKLEARN_SERVER
        modelUri: https://datalake555.blob.core.windows.net/mycontainer/sklearn/model.joblib
        name: classifier
      name: default
      replicas: 1
EOF

Logged into "https://openshift.default.svc.cluster.local:443" as "kube:admin" using the token provided.

You have access to 68 projects, the list has been suppressed. You can list all projects with ' projects'

Using project "default".
seldondeployment.machinelearning.seldon.io/example created


In [64]:
%%bash

# Test model in Seldon
MODEL_URL=example-default.odh.svc.cluster.local:8000
curl -X POST $MODEL_URL/api/v1.0/predictions \
    -H 'Content-Type: application/json' \
    -d '{ "data": { "ndarray": [[1,2,3,4]] } }'

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.035372518282028136,0.006231902938911814,0.95839557877906]]},"meta":{"requestPath":{"classifier":"registry.connect.redhat.com/seldonio/sklearnserver@sha256:1155a06aa8903e8436aaccbad64bd864a4a480b625466f6033dd08af7748d42a"}}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   313  100   275  100    38   2024    279 --:--:-- --:--:-- --:--:--  2037
